In [2]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model, model_from_json
from tensorflow.keras.models import load_model, save_model
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError, SparseCategoricalCrossentropy
from tensorflow.keras.metrics import RootMeanSquaredError, Accuracy
from tensorflow.keras.optimizers import Adam
import DataFunctions as data
from tcn import tcn
import tensorflowjs as tfjs

2024-05-19 16:04:15.068231: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-19 16:04:15.123688: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-19 16:04:15.123723: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-19 16:04:15.125594: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-19 16:04:15.135579: I tensorflow/core/platform/cpu_feature_guar

In [17]:
#   USER 1 BAG

#Se obtienen los datos de los sensores normalizados y quitando los ceros. También se obtienen las etiquetas
print("Obteniendo datos...")
data_070717_bag, labels_070717_bag = data.get_data('Data10Tensors/User3/User3_10tensors_070717_bag.csv', '../User3/070717/Label_New_User3_070717.txt')

print(len(data_070717_bag))
print(len(labels_070717_bag))

# Del dataset de datos se tienen que sacar 10 arrays numpy, uno por columna
df_arrays_070717_bag, dict_arrays_070717_bag = data.get_arrays_from_data(data_070717_bag, labels_070717_bag)
# Dividimos cada array en ventanas
windows_dict_070717_bag, windows_labels_070717_bag = data.divide_data_arrays_into_windows(df_arrays_070717_bag, labels_070717_bag, True, 500)
print(len(windows_dict_070717_bag['AcelX'][0]))
print(windows_dict_070717_bag['AcelX'][:3])
print(len(windows_dict_070717_bag['AcelX'][0]))
print(windows_labels_070717_bag[:-11])

# Dividimos los conjuntos de ventanas en los subconjuntos de entrenamiento y pruebas
x_train_070717_bag, y_train_070717_bag, x_test_070717_bag, y_test_070717_bag = data.divide_train_test(windows_dict_070717_bag, windows_labels_070717_bag, 0.7)
print("Tamaño x_train --> ", x_train_070717_bag['AcelX'].shape)
print("Tamaño y_train --> ", y_train_070717_bag.shape)
print("Tamaño x_test --> ", x_test_070717_bag['AcelX'].shape)
print("Tamaño y_test --> ", y_test_070717_bag.shape)
print(np.unique(np.array(y_train_070717_bag)))

input_tensors_070717_bag = [tf.constant(x_train_070717_bag['AcelX']), tf.constant(x_train_070717_bag['AcelY']), tf.constant(x_train_070717_bag['AcelZ']), tf.constant(x_train_070717_bag['MagX']), tf.constant(x_train_070717_bag['MagY']), tf.constant(x_train_070717_bag['MagZ']), tf.constant(x_train_070717_bag['GirosX']), tf.constant(x_train_070717_bag['GirosY']), tf.constant(x_train_070717_bag['GirosZ']), tf.constant(x_train_070717_bag['Pres'])]
print(len(input_tensors_070717_bag))
input_tensors_test_070717_bag = [tf.constant(x_test_070717_bag['AcelX']), tf.constant(x_test_070717_bag['AcelY']), tf.constant(x_test_070717_bag['AcelZ']), tf.constant(x_test_070717_bag['MagX']), tf.constant(x_test_070717_bag['MagY']), tf.constant(x_test_070717_bag['MagZ']), tf.constant(x_test_070717_bag['GirosX']), tf.constant(x_test_070717_bag['GirosY']), tf.constant(x_test_070717_bag['GirosZ']), tf.constant(x_test_070717_bag['Pres'])]
print(len(input_tensors_test_070717_bag))

input_shape_070717_bag = x_train_070717_bag['AcelX'].shape[1:]
print(input_shape_070717_bag)
model_070717_bag = data.create_model(input_shape_070717_bag)

model_070717_bag.fit(input_tensors_070717_bag, tf.cast(tf.constant(y_train_070717_bag), tf.float32), epochs=100)

model_070717_bag.evaluate(x=input_tensors_test_070717_bag, y=y_test_070717_bag)

Obteniendo datos...
2772069
2772069
500
[[0.50053443 0.50016916 0.4985727  ... 0.55339297 0.52323788 0.50785172]
 [0.52055949 0.5641143  0.66144441 ... 0.77176262 0.77155477 0.80733289]
 [0.87316238 0.95551693 1.04186681 ... 1.03392188 0.92344196 0.73966601]]
500
[2 2 2 ... 5 5 5]
Tamaño x_train -->  (3880, 500, 1)
Tamaño y_train -->  (3880,)
Tamaño x_test -->  (1664, 500, 1)
Tamaño y_test -->  (1664,)
[1 2 3 6]
10
10
(500, 1)
Input shape (500, 1)
Capa input --> (None, 500, 1)
Capa TCN ---> (None, 500, 32)
Capa MaxPooling ---> (None, 250, 32)
Capa de normalizacion ---> (None, 250, 32)
Capa multi atención ---> (None, 250, 32)
Capa de MHA --> (None, 250, 320)
Capa de convolución --> (None, 250, 64)
Capa de MaxPooling --> (None, 125, 64)
Capa de GAPooling --> (None, 64)
Capa de salida de la red --> (None, 8)
Epoch 1/100
122/122 [==============================] - 74s 140ms/step - loss: 0.1702 - accuracy: 0.9660
Epoch 2/100
122/122 [==============================] - 16s 131ms/step - loss: 0

[18.54119873046875, 0.16045673191547394]

In [18]:
model_json = model_070717_bag.to_json()
with open('ModelsMemo/User3/user3_070717_bag/model.json', "w") as json_file:
    json_file.write(model_json)
model_070717_bag.save_weights('ModelsMemo/User3/user3_070717_bag/weights.h5')

In [ ]:
#   USER 1 hand

#Se obtienen los datos de los sensores normalizados y quitando los ceros. También se obtienen las etiquetas
print("Obteniendo datos...")
data_070717_hand, labels_070717_hand = data.get_data('Data10Tensors/User3/User3_10tensors_070717_hand.csv', '../User3/070717/Label_New_User3_070717_v2.txt')

print(len(data_070717_hand))
print(len(labels_070717_hand))

# Del dataset de datos se tienen que sacar 10 arrays numpy, uno por columna
df_arrays_070717_hand, dict_arrays_070717_hand = data.get_arrays_from_data(data_070717_hand, labels_070717_hand)
# Dividimos cada array en ventanas
windows_dict_070717_hand, windows_labels_070717_hand = data.divide_data_arrays_into_windows(df_arrays_070717_hand, labels_070717_hand, True, 500)
print(len(windows_dict_070717_hand['AcelX'][0]))
print(windows_dict_070717_hand['AcelX'][:3])
print(len(windows_dict_070717_hand['AcelX'][0]))
print(windows_labels_070717_hand[:-11])

# Dividimos los conjuntos de ventanas en los subconjuntos de entrenamiento y pruebas
x_train_070717_hand, y_train_070717_hand, x_test_070717_hand, y_test_070717_hand = data.divide_train_test(windows_dict_070717_hand, windows_labels_070717_hand, 0.7)
print("Tamaño x_train --> ", x_train_070717_hand['AcelX'].shape)
print("Tamaño y_train --> ", y_train_070717_hand.shape)
print("Tamaño x_test --> ", x_test_070717_hand['AcelX'].shape)
print("Tamaño y_test --> ", y_test_070717_hand.shape)
print(np.unique(np.array(y_train_070717_hand)))

input_tensors_070717_hand = [tf.constant(x_train_070717_hand['AcelX']), tf.constant(x_train_070717_hand['AcelY']), tf.constant(x_train_070717_hand['AcelZ']), tf.constant(x_train_070717_hand['MagX']), tf.constant(x_train_070717_hand['MagY']), tf.constant(x_train_070717_hand['MagZ']), tf.constant(x_train_070717_hand['GirosX']), tf.constant(x_train_070717_hand['GirosY']), tf.constant(x_train_070717_hand['GirosZ']), tf.constant(x_train_070717_hand['Pres'])]
print(len(input_tensors_070717_hand))
input_tensors_test_070717_hand = [tf.constant(x_test_070717_hand['AcelX']), tf.constant(x_test_070717_hand['AcelY']), tf.constant(x_test_070717_hand['AcelZ']), tf.constant(x_test_070717_hand['MagX']), tf.constant(x_test_070717_hand['MagY']), tf.constant(x_test_070717_hand['MagZ']), tf.constant(x_test_070717_hand['GirosX']), tf.constant(x_test_070717_hand['GirosY']), tf.constant(x_test_070717_hand['GirosZ']), tf.constant(x_test_070717_hand['Pres'])]
print(len(input_tensors_test_070717_hand))

input_shape_070717_hand = x_train_070717_hand['AcelX'].shape[1:]
print(input_shape_070717_hand)
model_070717_hand = data.create_model(input_shape_070717_hand)

model_070717_hand.fit(input_tensors_070717_hand, tf.cast(tf.constant(y_train_070717_hand), tf.float32), epochs=100)

model_070717_hand.evaluate(x=input_tensors_test_070717_hand, y=y_test_070717_hand)

In [20]:
model_json = model_070717_hand.to_json()
with open('ModelsMemo/User3/user3_070717_hand/model.json', "w") as json_file:
    json_file.write(model_json)
model_070717_hand.save_weights('ModelsMemo/User3/user3_070717_hand/weights.h5')

In [ ]:
#   USER 1 HIPS

#Se obtienen los datos de los sensores normalizados y quitando los ceros. También se obtienen las etiquetas
print("Obteniendo datos...")
data_070717_hips, labels_070717_hips = data.get_data('Data10Tensors/User3/User3_10tensors_070717_hips.csv', '../User3/070717/Label_New_User3_070717.txt')

print(len(data_070717_hips))
print(len(labels_070717_hips))

# Del dataset de datos se tienen que sacar 10 arrays numpy, uno por columna
df_arrays_070717_hips, dict_arrays_070717_hips = data.get_arrays_from_data(data_070717_hips, labels_070717_hips)
# Dividimos cada array en ventanas
windows_dict_070717_hips, windows_labels_070717_hips = data.divide_data_arrays_into_windows(df_arrays_070717_hips, labels_070717_hips, True, 500)
print(len(windows_dict_070717_hips['AcelX'][0]))
print(windows_dict_070717_hips['AcelX'][:3])
print(len(windows_dict_070717_hips['AcelX'][0]))
print(windows_labels_070717_hips[:-11])

# Dividimos los conjuntos de ventanas en los subconjuntos de entrenamiento y pruebas
x_train_070717_hips, y_train_070717_hips, x_test_070717_hips, y_test_070717_hips = data.divide_train_test(windows_dict_070717_hips, windows_labels_070717_hips, 0.7)
print("Tamaño x_train --> ", x_train_070717_hips['AcelX'].shape)
print("Tamaño y_train --> ", y_train_070717_hips.shape)
print("Tamaño x_test --> ", x_test_070717_hips['AcelX'].shape)
print("Tamaño y_test --> ", y_test_070717_hips.shape)
print(np.unique(np.array(y_train_070717_hips)))

input_tensors_070717_hips = [tf.constant(x_train_070717_hips['AcelX']), tf.constant(x_train_070717_hips['AcelY']), tf.constant(x_train_070717_hips['AcelZ']), tf.constant(x_train_070717_hips['MagX']), tf.constant(x_train_070717_hips['MagY']), tf.constant(x_train_070717_hips['MagZ']), tf.constant(x_train_070717_hips['GirosX']), tf.constant(x_train_070717_hips['GirosY']), tf.constant(x_train_070717_hips['GirosZ']), tf.constant(x_train_070717_hips['Pres'])]
print(len(input_tensors_070717_hips))
input_tensors_test_070717_hips = [tf.constant(x_test_070717_hips['AcelX']), tf.constant(x_test_070717_hips['AcelY']), tf.constant(x_test_070717_hips['AcelZ']), tf.constant(x_test_070717_hips['MagX']), tf.constant(x_test_070717_hips['MagY']), tf.constant(x_test_070717_hips['MagZ']), tf.constant(x_test_070717_hips['GirosX']), tf.constant(x_test_070717_hips['GirosY']), tf.constant(x_test_070717_hips['GirosZ']), tf.constant(x_test_070717_hips['Pres'])]
print(len(input_tensors_test_070717_hips))

input_shape_070717_hips = x_train_070717_hips['AcelX'].shape[1:]
print(input_shape_070717_hips)
model_070717_hips = data.create_model(input_shape_070717_hips)

model_070717_hips.fit(input_tensors_070717_hips, tf.cast(tf.constant(y_train_070717_hips), tf.float32), epochs=100)

model_070717_hips.evaluate(x=input_tensors_test_070717_hips, y=y_test_070717_hips)

In [22]:
model_json = model_070717_hips.to_json()
with open('ModelsMemo/User3/user3_070717_hips/model.json', "w") as json_file:
    json_file.write(model_json)
model_070717_hips.save_weights('ModelsMemo/User3/user3_070717_hips/weights.h5')